In [1]:

import ase
import numpy as np
from ase.visualize import view
from ase.io import read, write
from ase.io import cif, vasp, lammpsdata
from ase.build import make_supercell, sort, add_adsorbate, add_vacuum
import nglview

In [2]:
from ase.io.proteindatabank import read_proteindatabank

In [3]:
import os

In [4]:
pdb = [f for f in os.listdir() if '.pdb' in f]
pdb

['clean_2_4000_O.pdb']

In [5]:
assert len(pdb) == 1

In [6]:
pdb = pdb[0]

In [7]:
tokens = pdb.split("_")
n_oxygens = tokens[2]
n_oxygens = int(n_oxygens)

In [8]:
impactors_pdb = read_proteindatabank(pdb)
total = len(impactors_pdb)

In [9]:
impactors_pdb.set_atomic_numbers([13]*(total-n_oxygens) + [8]*n_oxygens)

In [10]:
v = view(impactors_pdb, viewer='ngl')
v

In [11]:
impactors_pdb.get_chemical_formula()

'Al25000O4000'

In [12]:
from ase.build import bulk

In [13]:
NiPrimitive = bulk('Ni', 'fcc', a=3.528, orthorhombic=True)
NiPrimitive.cell

Cell([2.4946727240261395, 2.4946727240261395, 3.528])

In [14]:
NiPrimitive_a_b = NiPrimitive.cell[0][0]
NiPrimitive_c = NiPrimitive.cell[2][2]

In [15]:
v = view(NiPrimitive, viewer='ngl')
v

In [16]:
target_height  = 101.8
repeats_ni_a_b =  round(target_height/NiPrimitive_a_b)
repeats_ni_a_b

41

In [17]:
target_width_ni = 56
repeats_ni_c = 2*round(target_width_ni/NiPrimitive_c)
repeats_ni_c

32

In [18]:
AlPrimitive = bulk('Al', 'fcc', a=4.064, orthorhombic=True)
AlPrimitive.cell

Cell([2.873681958742129, 2.873681958742129, 4.064])

In [19]:
AlPrimitive_a_b = AlPrimitive.cell[0][0]
AlPrimitive_c = AlPrimitive.cell[2][2]

In [20]:
repeats_al_a_b =  round(target_height/AlPrimitive_a_b)
repeats_al_a_b

35

In [21]:
target_width_al = 28
repeats_al_c = 2*round(target_width_al/AlPrimitive_c)
repeats_al_c

14

In [22]:
nisalb = ase.build.fcc100("Ni", (repeats_ni_a_b, repeats_ni_a_b, repeats_ni_c), a=3.528, vacuum=1.0, orthogonal=True, periodic=True)

In [23]:
nisalb.cell

Cell([102.28158168507174, 102.28158168507174, 56.684])

In [24]:
v = view(nisalb, viewer='ngl')
v

In [25]:
alsalb = ase.build.fcc100("Al", (repeats_al_a_b, repeats_al_a_b, repeats_al_c), a=4.064, vacuum=1.0, orthogonal=True, periodic=True)

In [26]:
alsalb.cell

Cell([100.57886855597452, 100.57886855597452, 28.416])

In [27]:
v = view(alsalb, viewer='ngl')
v

In [28]:
alsalb_half  = ase.build.fcc100("Al", (repeats_al_a_b, repeats_al_a_b, repeats_al_c//2), a=4.064, vacuum=1.0, orthogonal=True, periodic=True)

In [29]:
wafer = ase.build.stack(alsalb, nisalb, maxstrain=None)
cap_left = ase.build.stack(alsalb_half, nisalb, maxstrain=None)

In [30]:
reverse_wafer = ase.build.stack(nisalb,alsalb, maxstrain=None)
cap_right = ase.build.stack(nisalb,alsalb_half, maxstrain=None)

In [31]:
v = view(wafer, viewer='ngl')
v

In [32]:
v = view(reverse_wafer, viewer='ngl')
v

In [33]:
combined_wafer_left = ase.build.stack(wafer,wafer, maxstrain=None)
combined_wafer_left = ase.build.stack(cap_left,combined_wafer_left, maxstrain=None)

In [34]:
v = view(combined_wafer_left, viewer='ngl')
v

In [35]:
combined_wafer_right = ase.build.stack(reverse_wafer,reverse_wafer, maxstrain=None)
combined_wafer_right = ase.build.stack(combined_wafer_right,cap_right, maxstrain=None)

In [36]:
v = view(combined_wafer_right, viewer='ngl')
v

In [37]:
combined_including_al2O3 = ase.build.stack(combined_wafer_left,impactors_pdb, maxstrain=None)


In [38]:
v = view(combined_including_al2O3, viewer='ngl')
v

In [39]:
combined_including_al2O3_both_sides = ase.build.stack(combined_including_al2O3,combined_wafer_right, maxstrain=None)


In [40]:
v = view(combined_including_al2O3_both_sides, viewer='ngl')
v

In [41]:

lammpsdata.write_lammps_data("full_slab.data", combined_including_al2O3_both_sides)
combined_including_al2O3_both_sides.write("full_slab.xyz")

/home/hmacdope/anaconda3/envs/terry/lib/python3.9/site-packages/ase/io/extxyz.py:302: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


In [42]:
formula = combined_including_al2O3_both_sides.get_chemical_formula()
formula

'Al110750Ni322752O4000'

In [43]:
import re
res = re.findall('[0-9]+', formula)

In [44]:
Al = int(res[0])
Ni = int(res[1])
O = int(res[2])

In [45]:
Ni/Al

2.9142392776523702